In [12]:
import pickle
import chess
import pandas as pd
import numpy as np 
from chess_minimax2 import get_minimax_move
from chess_random import get_random_move
from chess_neuralnet import get_NN_move

In [13]:
with open('2014_08_over_2400_chess_data.pickle','rb') as read_file:
    chess_df = pickle.load(read_file)

chess_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106 entries, Game47674 to Game993220
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BlackElo       106 non-null    object
 1   WhiteElo       106 non-null    object
 2   moves          106 non-null    object
 3   move_num       106 non-null    object
 4   winner         106 non-null    object
 5   Black_Elo_Num  106 non-null    int64 
 6   White_Elo_Num  106 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 6.6+ KB


In [14]:
chess_df.reset_index(inplace=True)
moves = chess_df['moves']

In [15]:
char_list = []
for i in range(1,500):
    char_list.append(str(i) + '.')

move_list = []

for game in moves:
    index = list(moves).index(game)
    allmoves = game.split()
    all_moves = [elem for elem in allmoves if elem not in char_list]
    board = chess.Board()
    for i in range(len(all_moves)):
        board.push_san(all_moves[i])
        fen = board.fen()
        row = [index, i, fen, all_moves[i]]
        move_list.append(row)

In [16]:
move_df = pd.DataFrame(move_list, columns = ['Game_Num', 'Move_Num', 'FEN', 'Move'])
move_df['Next_Move'] = move_df.Move.shift(-1)
ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move'].apply(lambda x : x[:-1])

<ipython-input-16-00a410d3eefe>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move'].apply(lambda x : x[:-1])


In [17]:
ddf['Rand_Move'] =ddf['FEN'].apply(get_random_move)
ddf['Minimax_Move'] =ddf['FEN'].apply(get_minimax_move)
ddf['NeuralNet_Move'] =ddf['FEN'].apply(get_NN_move)
ddf.head()

KeyboardInterrupt: 

In [ ]:
ddf['Random_Compare'] = np.where(ddf['Next_Move'] == ddf['Rand_Move'], True, False)
ddf['Minimax_Compare'] = np.where(ddf['Next_Move'] == ddf['Minimax_Move'], True, False)
ddf['NeuralNet_Compare'] = np.where(ddf['Next_Move'] == ddf['NeuralNet_Move'], True, False)

In [ ]:
ddf['Random_Compare'].value_counts()

In [ ]:
ddf['Minimax_Compare'].value_counts()

In [ ]:
ddf['NeuralNet_Compare'].value_counts()